In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA silver;

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.silver.sales_facts
USING DELTA
AS
SELECT
    DATE(event_time)     AS event_date,   -- derived
    event_time,
    user_id,
    product_id,
    category_code,
    event_type,
    price
FROM ecommerce.bronze.events
WHERE event_type IN ('view','cart','purchase');

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT event_date, COUNT(*) 
FROM ecommerce.silver.sales_facts
GROUP BY event_date
ORDER BY event_date;

event_date,COUNT(*)
2019-10-01,1244245
2019-10-02,1191328
2019-10-03,1127303
2019-10-04,1417190
2019-10-05,1330339
2019-10-06,1318379
2019-10-07,1200531
2019-10-08,1370633
2019-10-09,1347543
2019-10-10,1284077


In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold.revenue_by_category AS
SELECT
    category_code,
    SUM(price) AS total_revenue
FROM ecommerce.silver.sales_facts
WHERE event_type = 'purchase'
GROUP BY category_code;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold.product_conversion AS
SELECT
    product_id,
    SUM(CASE WHEN event_type='purchase' THEN 1 ELSE 0 END) * 1.0 /
    SUM(CASE WHEN event_type='view' THEN 1 ELSE 0 END) AS conversion_rate
FROM ecommerce.silver.sales_facts
GROUP BY product_id;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold.daily_purchases AS
SELECT
    event_date,
    COUNT(*) AS purchases
FROM ecommerce.silver.sales_facts
WHERE event_type='purchase'
GROUP BY event_date;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold.view_no_purchase AS
SELECT DISTINCT v.user_id
FROM ecommerce.silver.sales_facts v
LEFT ANTI JOIN ecommerce.silver.sales_facts p
ON v.user_id = p.user_id
AND p.event_type = 'purchase'
WHERE v.event_type = 'view';


num_affected_rows,num_inserted_rows
